In [1]:
example1 = """
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
"""

In [2]:
import itertools
import re
from pprint import pprint

import numpy as np

data = open("input.txt").read()

In [4]:
!pip install numpy

In [5]:
def read_data(data):
    for line in data.strip().split("\n"):
        yield [list(map(int, m.groups())) for m in re.finditer(r"(\d+),(\d+)", line)]


def grow_array(arr, new_shape):
    if arr.shape == new_shape:
        return arr
    new_arr = np.zeros(new_shape, dtype=arr.dtype)
    new_arr[: arr.shape[0], : arr.shape[1]] = arr
    return new_arr


def accomodate(arr, x1, y1, x2, y2):
    max_x = max(x1, x2, arr.shape[1] - 1)
    max_y = max(y1, y2, arr.shape[0] - 1)
    return grow_array(arr, (max_y + 1, max_x + 1))


def unending(x1, x2, n=0):
    d = -1 if x2 < x1 else 1
    return itertools.chain(
        range(x1, x2 + d, d),
        itertools.repeat(x2, n),
    )


def points(x1, y1, x2, y2):
    counts = abs(x1 - x2), abs(y1 - y2)
    n = int(abs(counts[0] - counts[1]))
    yield from zip(unending(x1, x2, n=n), unending(y1, y2, n=n))


def task1(data):
    lines = list(read_data(data))
    max_y = max(y for x, y in itertools.chain.from_iterable(lines))
    cave = np.array([[]], dtype=int, order="F")

    for line in lines:
        for src, dst in itertools.pairwise(line):
            cave = accomodate(cave, *src, *dst)
            for x, y in points(*src, *dst):
                cave[y][x] = 1

    for sand in itertools.count():
        sand_x, sand_y = 500, 0
        while sand_y < max_y:
            new_y = sand_y + 1
            for new_x in (sand_x, sand_x - 1, sand_x + 1):
                if not cave[new_y][new_x]:
                    sand_x, sand_y = new_x, new_y
                    break
            else:
                break
        if sand_y >= max_y:
            break
        cave[sand_y][sand_x] = 2
    return sand


task1(example1)

24

In [6]:
task1(data)

832

In [7]:
def print_cave(cave):
    min_x = min(
        next((i for i, v in enumerate(row) if v), cave.shape[1]) for row in cave
    )
    t = {0: ".", 1: "#", 2: "o"}
    for row in cave:
        print("".join(t[c] for c in row[min_x:]).rstrip("."))


def task2(data):
    lines = list(read_data(data))
    max_y = max(y for x, y in itertools.chain.from_iterable(lines))
    max_x = max(x for x, y in itertools.chain.from_iterable(lines))
    cave = np.zeros((max_y + 2, max_x * 3), dtype=int, order="F")

    for line in lines:
        for src, dst in itertools.pairwise(line):
            for x, y in points(*src, *dst):
                cave[y][x + max_x] = 1
    print_cave(cave)

    for sand in itertools.count():
        sand_x, sand_y = 500 + max_x, 0
        while sand_y < max_y + 1:
            new_y = sand_y + 1
            for new_x in (sand_x, sand_x - 1, sand_x + 1):
                if not cave[new_y][new_x]:
                    sand_x, sand_y = new_x, new_y
                    break
            else:
                break
        cave[sand_y][sand_x] = 2
        if sand_y == 0:
            break
    print_cave(cave)
    return sand + 1


task2(example1)  # >27600





....#...##
....#...#
..###...#
........#
........#
#########

..........o
.........ooo
........ooooo
.......ooooooo
......oo#ooo##o
.....ooo#ooo#ooo
....oo###ooo#oooo
...oooo.oooo#ooooo
..oooooooooo#oooooo
.ooo#########ooooooo
ooooo.......ooooooooo


93

In [8]:
task2(data)














..............................#
..............................#
..............................#...........#
..............................#...........#
..............................#.#.........#
..............................#.#.#.....#.#...#
..............................#.#.#.....#.#...#.#
..............................#.#.#.....#.#...#.#
..............................#.#.#...#.#.#...#.#
..............................#.#.#.#.#.#.#.#.#.#
..............................###################



.......................................#
.......................................#####################


..........................................................#...#
..........................................................#...#
..........................................................#...#
..........................................................#...#
.........................................................##...########
.........................................................#.

27601